$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\newcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\cset}[1]{\mathcal{#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bb}[1]{\boldsymbol{#1}}
\newcommand{\E}[2][]{\mathbb{E}_{#1}\left[#2\right]}
\newcommand{\ip}[3]{\left<#1,#2\right>_{#3}}
\newcommand{\given}[]{\,\middle\vert\,}
\newcommand{\DKL}[2]{\cset{D}_{\text{KL}}\left(#1\,\Vert\, #2\right)}
\newcommand{\grad}[]{\nabla}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
$$

# Part 2: Summary Questions
<a id=part2></a>

This section contains summary questions about various topics from the course material.

You can add your answers in new cells below the questions.

**Notes**

- Clearly mark where your answer begins, e.g. write "**Answer:**" in the beginning of your cell.
- Provide a full explanation, even if the question doesn't explicitly state so. We will reduce points for partial explanations!
- This notebook should be runnable from start to end without any errors.

### CNNs

1. Explain the meaning of the term "receptive field" in the context of CNNs.

**Answer:**

The receptive field of a unit (neuron) in a CNN is the spacial region of the input that has an affect on the unit. The receptive field of a unit that is not in the first hidden layer is the union of the recptive field of it's input, because it is affected by it. The higher the convolution kernel size, stride and dialation of a layer, the larger the receptive field is.
_____

2. Explain and elaborate about three different ways to control the rate at which the receptive field grows from layer to layer. Compare them to each other in terms of how they combine input features.

**Answer:**

a. *Kernel Size* - The size of the convolution kernel. At each layer, each unit is affected by an amount of units that depends on how large is the kernel size, thus, the larger values of kernel size we use in the network, the higher the rate in which the receptive field grows. 
The higher the kernel size is, the more features of the input affect each unit of the output, but the extracted features are less regional.


b. *Stirde* - Detemines how many input points are skipped between each step of the convolution. This value, has an affect on how an output unit will be similar to it's neighbours becasue they have common inputs. When this value is higher, neighbor units will depand on different inputs, thus when performing convolution again on the output, the next layer will have a higher receptive field.
The stride value, does not affect directly the receptive field of a layer but only the receptive field of the next layer.

c. *Dialation* - Determines the gap inconvolution within each step.  The higher the dialation, the larger the area the area of the input the kernel "looks at", but more input values are ignored between those who are used. Thats means of course that increasing the dialation also increases the receptive field, but some features from the input are lost. A dialation may be similar to pooling + convolution.
_____

3. Imagine a CNN with three convolutional layers, defined as follows:

In [ ]:
import torch
import torch.nn as nn

cnn = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=4, out_channels=16, kernel_size=5, stride=2, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=7, dilation=2, padding=3),
    nn.ReLU(),
)

cnn(torch.rand(size=(1, 3, 1024, 1024), dtype=torch.float32)).shape

What is the size (spatial extent) of the receptive field of each "pixel" in the output tensor?

**Answer:**

Every pixel after the first layer has recptive field of 3x3

# TODO!

4. You have trained a CNN, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$, and $f_l(\cdot;\vec{\theta}_l)$ is a convolutional layer (not including the activation function).

  After hearing that residual networks can be made much deeper, you decide to change each layer in your network you used the following residual mapping instead $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)+\vec{x}$, and re-train.

  However, to your surprise, by visualizing the learned filters $\vec{\theta}_l$ you observe that the original network and the residual network produce completely different filters. Explain the reason for this.

**Answer:** 
    
By creating residual connection in the network, we force the layer to learn the identity function plus another
function of the input in contrast to a regular layer where the network learns a completly new function of the input. therefore, 
with resudual connection the network is expected to learn different mapping, which will probably have smaller weights since it's
initial pretrained output is the input + random function, there fore we will get completly different filters.

### Dropout

1. Consider the following neural network:

In [ ]:
import torch.nn as nn

p1, p2 = 0.1, 0.2
nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Dropout(p=p1),
    nn.Dropout(p=p2),
)

If we want to replace the two consecutive dropout layers with a single one defined as follows:
```python
nn.Dropout(p=q)
```
what would the value of `q` need to be? Write an expression for `q` in terms of `p1` and `p2`.

**Answer:**

A value is droped out if it was droped out in the first dropout layer, the second one, or both. 
We will calculate the probability a value is not droped out:

$ 1 - q = (1-p_1)(1-p_2) $

$ q = 1 - (1 - p_2 - p_1 + p_1 p_2) = p_1 + p_2 - p_1 p_2$

2. **True or false**: dropout must be placed only after the activation function.

**Answer:**

Altough, as a rule of thump a dropout is suggested to be put after the activation layer, 
it can also be put before it, and in the case of a relu it is even more computationally efficient.

3. After applying dropout with a drop-probability of $p$, the activations are scaled by $1/(1-p)$. Prove that this scaling is required in order to maintain the value of each activation unchanged in expectation.

**Answer:**

Consider an activation vector $\mathbf{x}$, and the same activation vector $\mathbf{\tilde x}$ with dropout applied, which means each 

$\mathbf{\tilde x_i} = \begin{cases}
    x_i & \text{w.p } (1-p) \\ 
    0 & \text{w.p } (p)
\end{cases}$ 

Therefore 

$\mathbb{E}[\mathbf{\tilde x}] = 1/n \sum_{i=1}^{n}{\tilde x_i} = 1/n \sum_{i=1}^{n}{(1-p)x_i} = (1-p) \mathbb{E}[\mathbf{x}]$ 

The expectation of the activation is :
$\mathbb{E}[\sum{w_i \tilde x_i}] = (1-p) \mathbb{E}[\sum{w_i x_i}]$ therefore in order to maintain the expectation, we have to scale it by $1/(1-p)$

### Losses and Activation functions

1. You're training a an image classifier that, given an image, needs to classify it as either a dog (output 0) or a hotdog (output 1). Would you train this model with an L2 loss? if so, why? if not, demonstrate with a numerical example. What would you use instead?

2. After months of research into the origins of climate change, you observe the following result:

<center><img src="https://sparrowism.soc.srcf.net/home/piratesarecool4.gif" /></center>

You decide to train a cutting-edge deep neural network regression model, that will predict the global temperature based on the population of pirates in `N` locations around the globe.
You define your model as follows:

In [ ]:
import torch.nn as nn

N = 42  # number of known global pirate hot spots
H = 128
mlpirate = nn.Sequential(
    nn.Linear(in_features=N, out_features=H),
    nn.Sigmoid(),
    *[
        nn.Linear(in_features=H, out_features=H), nn.Sigmoid(),
    ]*24,
    nn.Linear(in_features=H, out_features=1),
)

While training your model you notice that the loss reaches a plateau after only a few iterations.
It seems that your model is no longer training.
What is the most likely cause?

3. Referring to question 2 above: A friend suggests that if you replace the `sigmoid` activations with `tanh`, it will solve your problem. Is he correct? Explain why or why not.

4. Regarding the ReLU activation, state whether the following sentences are **true or false** and explain:
  1. In a model using exclusively ReLU activations, there can be no vanishing gradients.
  1. The gradient of ReLU is linear with its input when the input is positive.
  1. ReLU can cause "dead" neurons, i.e. activations that remain at a constant value of zero.

### Optimization

1. Explain the difference between: stochastic gradient descent (SGD), mini-batch SGD and regular gradient descent (GD).

**Answer:**
    
The difference is in the amount of samples used to calculate the gradient in each step.
In regular gradient decent we use all of our training samples in each step, that may cause the model to overfit and also be to expensive to calculate, or even impossible due to memory.
In stochastic gradient decent we use one sample in each step, that may prevent the model from converging to a minimum because the updates are chaotic. 
In batch gradient decent we use a fixed small number (usualy 2-1024) of samples in each step.

2. Regarding SGD and GD:
  1. Provide at least two reasons for why SGD is used more often in practice compared to GD.
  2. In what cases can GD not be used at all?

**Answer:**

Two reasons: 

    a. Calculating the gradients for all the samples may take long time, so each gradient step would take long time and the model would train slow
    b. GD tends to overfit the training set while when using SGD the model may have better generalization.

GD can't be used at all when there are too mach data samples so the values of the gradients can't be all together in the memory at the same time.
  

3. You have trained a deep resnet to obtain SoTA results on ImageNet.
While training using mini-batch SGD with a batch size of $B$, you noticed that your model converged to a loss value of $l_0$ within $n$ iterations (batches across all epochs) on average.
Thanks to your amazing results, you secure funding for a new high-powered server with GPUs containing twice the amount of RAM.
You're now considering to increase the mini-batch size from $B$ to $2B$.
Would you expect the number of of iterations required to converge to $l_0$ to decrease or increase when using the new batch size? explain in detail.

**Answer:**

We would expect less iterations to converge. Since we are using more samples in each update steps, the update dircetion would be more accurate and less stochastic, therfore we need less steps to converge since the optimizer moves to a better direction. Altough we would need less steps to converge, it will not necessarily take less time, because at each iteration we have to use more samples to calculate the loss.

4. For each of the following statements, state whether they're **true or false** and explain why.
  1. When training a neural network with SGD, every epoch we perform an optimization step for each sample in our dataset.
  1. Gradients obtained with SGD have less variance and lead to quicker convergence compared to GD.
  1. SGD is less likely to get stuck in local minima, compared to GD.
  1. Training  with SGD requires more memory than with GD.
  1. Assuming appropriate learning rates, SGD is guaranteed to converge to a local minimum, while GD is guaranteed to converge to the global minimum.
  1. Given a loss surface with a narrow ravine (high curvature in one direction): SGD with momentum will converge more quickly than Newton's method which doesn't have momentum.

**Answers:**

5. False, in SGD we sample randomly from the training set at each step, some samples may be sampled twice or more in that proccess while other are not sampled at all

6. False, gradients in SGD are only affected by one sample. samples may be different therefore the variance will be higher and convergence will be slower

7. True, since in SGD different samples are used in each step, new samples may get the optimizer out of a local minima that caused by other samples

8. False, in SGD we don't have to use all samples simultaneously, so we dont have to keep them all in the memory at the same time

9. False, GD is not guaranteed to converge to a global minimum. it depends on the initialization.

10. True, in Newton's method we  will have to compute the second order derivitives for the hessian, which will take time in each step, while with momentum we will only have to compute the gradient. 

5. In tutorial 5 we saw an example of bi-level optimization in the context of deep learning, by embedding an optimization problem as a layer in the network.
  **True or false**: In order to train such a network, the inner optimization problem must be solved with a descent based method (such as SGD, LBFGS, etc).
  Provide a mathematical justification for your answer.

**Answer**:
    
לא הבנתי כלכך, אני בנתיים מעתיק תשובה מרפרנס שמצאתי ולא הבנתי:

    False, The inner optimization problem can be solved as a constraint in the network's loss function. Therefore we can minimize an expression $L = f(x) + \lambda g(x)$ without using a descent based method just like we find a minimum on a regular function.

6. You have trained a neural network, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$ for some arbitrary parametrized functions $f_l(\cdot;\vec{\theta}_l)$.
  Unfortunately while trying to break the record for the world's deepest network, you discover that you are unable to train your network with more than $L$ layers.
  1. Explain the concepts of "vanishing gradients", and "exploding gradients".
  2. How can each of these problems be caused by increased depth?
  3. Provide a numerical example demonstrating each.
  4. Assuming your problem is either of these, how can you tell which of them it is without looking at the gradient tensor(s)?

**Answers:**

7. In vanishing gradients, the gradients goes smaller as they propogate through the network until they get close to zero and act like a numeric zero. In exploding gradients, the gradients glow's as they propogate until they get to big, thus the update step is to large so the optimizer can't find the minimum.

8. When back propogating the gradient, we multiply the gradient of the previus layer with the gradient of the current layer (according to the chain rule). Increasing the number of layer, will increasce the number of multipications. For gradients lower then 1, the values exponentialy decay to zero, and for gradients higher then 1 the values would exponentialy grow (to infinity)

9. Consider a netwrork with one unit in each layer, so the function it represents is: $\hat y = f_n(f_{n-1}(...f_1(x;\Theta_1)...;\Theta_{n-1});\Theta_n))$ for functions $f_i$ parametrized by $\Theta_i$ as the layers.

    for vanishing gradients, let's assume that $\frac{\partial L}{ \partial \Theta_n} = \frac{\partial f_i}{\partial \Theta_i} = 0.9$ Which is a little bit lower then 1.
    Therfore, $\frac{\partial L}{ \partial \Theta_1} = \frac{\partial L}{ \partial \Theta_n} \frac{\partial f_{n-1} }{ \partial \Theta_{n-1}} ... \frac{\partial f_{1} }{ \partial \Theta_{1}} = 0.9^n$. We can see an exponential decay with $n$ which is the number of layer. With 1,000 layers for example, we would get a gradient as low as $10^{-46}$ which is practicly zero.  
    
    or exploding gradients, let's assume that $\frac{\partial L}{ \partial \Theta_n} = \frac{\partial f_i}{\partial \Theta_i} = 1.1$ Which is a little bit higher then 1.
    Therfore, $\frac{\partial L}{ \partial \Theta_1} = \frac{\partial L}{ \partial \Theta_n} \frac{\partial f_{n-1} }{ \partial \Theta_{n-1}} ... \frac{\partial f_{1} }{ \partial \Theta_{1}} = 1.1^n$. We can see an exponential growth with $n$. With 1,000 layers, we would get a gradient as high as $10^{1}$ which is practicly infinity.  
    
10. We can look at the graph of the loss function. If the loss fluctuates, or diverges, it is probably becasue the gradients are too high, making the optimizer to take too large steps. If the loss is constant and does not change after a lot of steps, the steps are probably too small becasue the gradient is close to zero.

### Backpropagation

1. You wish to train the following 2-layer MLP for a binary classification task:
  $$
  \hat{y}^{(i)} =\mat{W}_2~ \varphi(\mat{W}_1 \vec{x}^{(i)}+ \vec{b}_1) + \vec{b}_2
  $$
  Your wish to minimize the in-sample loss function is defined as
  $$
  L_{\mathcal{S}} = \frac{1}{N}\sum_{i=1}^{N}\ell(y^{(i)},\hat{y}^{(i)}) + \frac{\lambda}{2}\left(\norm{\mat{W}_1}_F^2 + \norm{\mat{W}_2}_F^2 \right)
  $$
  Where the pointwise loss is binary cross-entropy:
  $$
  \ell(y, \hat{y}) =  - y \log(\hat{y}) - (1-y) \log(1-\hat{y})
  $$
  
  Write an analytic expression for the derivative of the final loss $L_{\mathcal{S}}$ w.r.t. each of the following tensors: $\mat{W}_1$, $\mat{W}_2$, $\mat{b}_1$, $\mat{b}_2$, $\mat{x}$.

2. The derivative of a function $f(\vec{x})$ at a point $\vec{x}_0$ is
  $$
  f'(\vec{x}_0)=\lim_{\Delta\vec{x}\to 0} \frac{f(\vec{x}_0+\Delta\vec{x})-f(\vec{x}_0)}{\Delta\vec{x}}
  $$
  
  1. Explain how this formula can be used in order to compute gradients of neural network parameters numerically, without automatic differentiation (AD).
  
  2. What are the drawbacks of this approach? List at least two drawbacks compared to AD.

3. Given the following code snippet:
  1. Write a short snippet that implements that calculates gradient of `loss` w.r.t. `W` and `b` using the approach of numerical gradients from the previous question.
  2. Calculate the same derivatives with autograd.
  3. Show, by calling `torch.allclose()` that your numerical gradient is close to autograd's gradient.

In [ ]:
import torch

N, d = 100, 5
dtype = torch.float64
X = torch.rand(N, d, dtype=dtype)
W, b = torch.rand(d, d, requires_grad=True, dtype=dtype), torch.rand(d, requires_grad=True, dtype=dtype)

def foo(W, b):
    return torch.mean(X @ W + b)

loss = foo(W, b)
print(f"{loss=}")

# TODO: Calculate gradients numerically for W and b
# grad_W =...
# grad_b =...

# TODO: Compare with autograd using torch.allclose()
# autograd_W = ...
# autograd_b = ...
# assert torch.allclose(grad_W, autograd_W)
# assert torch.allclose(grad_b, autograd_b)

### Sequence models

1. Regarding word embeddings:
  1. Explain this term and why it's used in the context of a language model.
  1. Can a language model like the sentiment analysis example from the tutorials be trained without an embedding (i.e. trained directly on sequences of tokens)? If yes, what would be the consequence for the trained model? if no, why not?

2. Considering the following snippet, explain:
  1. What does `Y` contain? why this output shape?
  2. How you would implement `nn.Embedding` yourself using only torch tensors. 

In [ ]:
import torch.nn as nn

X = torch.randint(low=0, high=42, size=(5, 6, 7, 8))
embedding = nn.Embedding(num_embeddings=42, embedding_dim=42000)
Y = embedding(X)
print(f"{Y.shape=}")

3. Regarding truncated backpropagation through time (TBPTT) with a sequence length of S: State whether the following sentences are **true or false**, and explain.
  1. TBPTT uses a modified version of the backpropagation algorithm.
  2. To implement TBPTT we only need to limit the length of the sequence provided to the model to length S.
  3. TBPTT allows the model to learn relations between input that are at most S timesteps apart.

### Attention

1. In tutorial 7 (part 2) we learned how to use attention to perform alignment between a source and target sequence in machine translation.
  1. Explain qualitatively what the addition of the attention mechanism between the encoder and decoder does to the hidden states that the encoder and decoder each learn to generate (for their language). How are these hidden states different from the model without attention?
  
  2. After learning that self-attention is gaining popularity thanks to the shiny new transformer models, you decide to change the model from the tutorial: instead of the queries being equal to the decoder hidden states, you use self-attention, so that the keys, queries and values are all equal to the encoder's hidden states (with learned projections). What influence do you expect this will have on the learned hidden states?


### Unsupervised learning

1. As we have seen, a variational autoencoder's loss is comprised of a reconstruction term and  a KL-divergence term. While training your VAE, you accidentally forgot to include the KL-divergence term.
What would be the qualitative effect of this on:

  1. Images reconstructed by the model during training ($x\to z \to x'$)?
  1. Images generated by the model ($z \to x'$)?

2. Regarding VAEs, state whether each of the following statements is **true or false**, and explain:
  1. The latent-space distribution generated by the model for a specific input image is $\mathcal{N}(\vec{0},\vec{I})$.
  2. If we feed the same image to the encoder multiple times, then decode each result, we'll get the same reconstruction.
  3. Since the real VAE loss term is intractable, what we actually minimize instead is it's upper bound, in the hope that the bound is tight.

2. Regarding GANs, state whether each of the following statements is **true or false**, and explain:
  1. Ideally, we want the generator's loss to be low, and the discriminator's loss to be high so that it's fooled well by the generator.
  2. It's crucial to backpropagate into the generator when training the discriminator.
  3. To generate a new image, we can sample a latent-space vector from $\mathcal{N}(\vec{0},\vec{I})$.
  4. It can be beneficial for training the generator if the discriminator is trained for a few epochs first, so that it's output isn't arbitrary.
  5. If the generator is generating plausible images and the discriminator reaches a stable state where it has 50% accuracy (for both image types), training the generator more will further improve the generated images.

### Graph Neural Networks

1. You have implemented a graph convolutional layer based on the following formula, for a graph with $N$ nodes:
$$
\mat{Y}=\varphi\left( \sum_{k=1}^{q} \mat{\Delta}^k \mat{X} \mat{\alpha}_k + \vec{b} \right).
$$
  1. Assuming $\mat{X}$ is the input feature matrix of shape $(N, M)$: what does $\mat{Y}$ contain in it's rows?
  1. Unfortunately, due to a bug in your calculation of the Laplacian matrix, you accidentally zeroed the row and column $i=j=5$ (assume more than 5 nodes in the graph).
What would be the effect of this bug on the output of your layer, $\mat{Y}$?

2. We have discussed the notion of a Receptive Field in the context of a CNN. How would you define a similar concept in the context of a GCN (i.e. a model comprised of multiple graph convolutional layers)?